# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [ ]:
from init import *
import semantic_kernel as sk

kernel = sk.KernelBuilder.create_kernel()

useAzureOpenAI = False
model = "text-davinci-003"

# Configure AI backend used by the kernel
if (useAzureOpenAI):
    api_key, endpoint = azure_openai_settings_from_dot_env()
    kernel.config.add_azure_openai_completion_backend("davinci", model, endpoint, api_key, overwrite=True)
else:
    api_key, org_id = openai_settings_from_dot_env()
    kernel.config.add_openai_completion_backend("davinci", model, api_key, org_id, overwrite=True)

Let's define a prompt outlining a dialogue chat bot.

In [ ]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
Human: {{$human_input}}
ChatBot:";
"""

prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=2000, temperature=0.7, top_p=0.5)

Register your semantic function

In [ ]:
prompt_template = sk.PromptTemplate(sk_prompt, kernel.prompt_template_engine, prompt_config)

function_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)
chat_function = kernel.register_semantic_function("ChatBot", "Chat", function_config)

Initialize your context

In [ ]:
context = sk.ContextVariables()

history = ""
context.set("history", history)

Chat with the Bot

In [ ]:
human_input = "Hi, I'm looking for book suggestions"
context.set("human_input", human_input)

bot_answer = await kernel.run_on_str_async(context, chat_function)
print(bot_answer)

Update the history with the output

In [ ]:
history += "\nHuman: {human_input}\nMelody: {bot_answer}\n"
context.update(history);

print(context)

Keep Chatting!

In [ ]:
history = ""
async def chat(input_text: str) -> str:
    global history
    # Save new message in the context variables
    context.set("human_input", input_text)

    # Process the user message and get an answer
    answer = await kernel.run_on_str_async(context, chat_function)

    # Append the new interaction to the chat history
    history = history + f"\nHuman: {input_text}\nMelody: {answer}\n"
    context.set("history", history)

    # Show the response
    print(context)

In [ ]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

In [ ]:
await chat("that sounds interesting, what is it about?")

In [ ]:
await chat("if I read that book, what exactly will I learn about Greece history?")

In [ ]:
await chat("could you list some more books I could read about this topic?")